In [64]:

import sys
sys.path.insert(0, '/Users/anis/test_labnbook/Indicators/scripts')  
import json, gzip,ujson
import pandas as pd 
from  utils.utils  import get_authors_changes
import numpy as np, toml
from typing import Tuple, List, Dict
import mysql.connector as mariadb
import warnings    
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows',700)

from flow_3 import * 
from utils.utils import *

password = "11950022"
host = "localhost"
database_name  = "Labnbook"
user = 'root'
try:
    conn = mariadb.connect(user=user, password=password,
                           host=host, database=database_name)
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")

# Récupération du curseur
cursor = conn.cursor()
trace = pd.read_sql(
    " SELECT id_trace, id_labdoc,id_user ,id_action, action_time from trace Order By id_labdoc ASC, action_time ASC", conn)
# WHERE  id_action=9 

In [81]:
def get_writing_time(id_labdoc : int, trace : pd.DataFrame)-> pd.DataFrame:
    """
    Calculates the number of modification (trace 9) for each change of authors and also multiplies it by 20 or 30 to have the effective writing time 
    output: a list of tuples (id_trace, author_id, num_modifications, writing_time).  
    ex: [(7481881, 12673, 23, 690),
                        (7482763, 12661, 4, 120),
                        (7482817, 12673, 14, 420),
                        (7482845, 12661, 2, 60),
                        [7505160, 12673, 58, 1740]]} 
    """ 
    #TODO: improve this code which is not optimal!
    labdoc  = trace[trace['id_labdoc'] == int(id_labdoc)]
    labdoc.reset_index(drop=True,inplace=True)
    if labdoc.empty:
        pass
    else:
        labdoc_time = labdoc["action_time"].iloc[0]
        date_to_compare = pd.Timestamp('2020-05-16')

        if labdoc_time.date() < date_to_compare.date():
            factor = 20
        else:
            factor = 30
        count_list = []
        current_author = None
        current_count = 0
        for author,id_trace in zip(labdoc['id_user'],labdoc["id_trace"]):
            if author == current_author:
                current_count += 1
            else:
                if current_author is not None:
                    idx  = labdoc[labdoc["id_trace"]==id_trace].index
                    
                    count_list.append((int(labdoc.loc[idx-1].id_trace.values[0]),id_labdoc,current_author, current_count,current_count*factor))
                current_author = author
                current_count = 1
        count_list.append([id_trace,id_labdoc,current_author, current_count,current_count*factor])
        
        return count_list

In [82]:
id_ld = 445494
count_list = get_writing_time(id_ld, trace)
labdoc = trace[trace["id_labdoc"]==id_ld]
index = get_authors_changes(list(labdoc["id_user"]))
ld = labdoc.iloc[index]
ld 

,id_trace,id_labdoc,id_user,id_action,action_time
7684794,7481881,445494.0,12673,10,2021-03-16 10:05:29
7684798,7482763,445494.0,12661,10,2021-03-16 11:18:11
7684812,7482817,445494.0,12673,10,2021-03-16 11:21:13
7684814,7482845,445494.0,12661,10,2021-03-16 11:22:42
7684872,7505160,445494.0,12673,10,2021-03-18 08:49:40


In [85]:
df1 = pd.DataFrame(count_list)
df2 =df1.head(2)

In [87]:
import pandas as pd

# créer deux dataframes
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [5, 6], 'B': [7, 8]})

# concaténer les deux dataframes en ligne
df_concat = pd.concat([df1, df2], axis=0,ignore_index= True)

print(df_concat)


   A  B
0  1  3
1  2  4
2  5  7
3  6  8
